In [ ]:
# Imports
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats

plt.rcParams['figure.figsize'] = (15.0, 8.0) # set default size of plots
plt.rcParams['figure.facecolor'] = 'white'

pd.set_option('display.max_rows', None)

matplotlib.rcParams.update({'font.size': 15})

In [ ]:
TIME_RETRIEVAL_REE = 0
TIME_RETRIEVAL_TEE = 10
TIME_RETRIEVAL_REE_WASM = 0
TIME_RETRIEVAL_TEE_WASM = 13

In [ ]:
def compute_mean_std(size, time, factor = 1):
    df = pd.DataFrame(columns = ["Size","Time"])
    df["Size"] = size
    df["Time"] = time * factor
    
    result = pd.DataFrame(columns = ["Size","Mean","Std"])
    result["Mean"] = df.groupby("Size")["Time"].median() #df.groupby("Size")["Time"].apply(stats.gmean)
    result["Std"] = df.groupby("Size")["Time"].std() #df.groupby("Size")["Time"].apply(stats.gstd)
    result["Size"] = df.groupby("Size").groups.keys()
    
    return result

def formatForLatex(n):
    roundedAsString = str(round(n, 0))
    if "." in roundedAsString:
        roundedAsString = roundedAsString.rstrip("0")
        roundedAsString = roundedAsString.rstrip(".")
    return f"{roundedAsString}"

In [ ]:
# Import the dataset
columns = ["Size","LaunchStart","MemoryStart","MemoryEnd","HashStart","HashEnd","InitEnd","LoadEnd","InstantiateEnd","LaunchEnd"]
launchtime_raw = pd.read_csv("../logs/launch-time/general.csv", names = columns)

launchtime_raw

In [ ]:
# Summary of the gathered timespans
launchtime = pd.DataFrame()
launchtime["Size"] = launchtime_raw["Size"]

launchtime["LaunchSpan"] = launchtime_raw["LaunchEnd"] - launchtime_raw["LaunchStart"] \
                        - TIME_RETRIEVAL_REE/2 - 7*TIME_RETRIEVAL_TEE - TIME_RETRIEVAL_TEE_WASM/2

launchtime["TransitionSpan"] = launchtime_raw["MemoryStart"] - launchtime_raw["LaunchStart"] \
                        - TIME_RETRIEVAL_REE/2 - TIME_RETRIEVAL_TEE/2

launchtime["MemorySpan"] = launchtime_raw["MemoryEnd"] - launchtime_raw["MemoryStart"] \
                        - 2*TIME_RETRIEVAL_TEE/2

launchtime["HashSpan"] = launchtime_raw["HashEnd"] - launchtime_raw["HashStart"] \
                        - 2*TIME_RETRIEVAL_TEE/2

launchtime["WasmInitSpan"] = launchtime_raw["InitEnd"] - launchtime_raw["HashEnd"] \
                        - 2*TIME_RETRIEVAL_TEE/2

launchtime["WasmLoadSpan"] = launchtime_raw["LoadEnd"] - launchtime_raw["InitEnd"] \
                        - 2*TIME_RETRIEVAL_TEE/2

launchtime["WasmInstantiateSpan"] = launchtime_raw["InstantiateEnd"] - launchtime_raw["LoadEnd"] \
                        - 2*TIME_RETRIEVAL_TEE/2

launchtime["WasmExecutionSpan"] = launchtime_raw["LaunchEnd"] - launchtime_raw["InstantiateEnd"] \
                        - TIME_RETRIEVAL_TEE/2 - TIME_RETRIEVAL_TEE_WASM/2

launchtime["OtherSpan"] = launchtime["LaunchSpan"] - launchtime["TransitionSpan"] - launchtime["MemorySpan"] \
                        - launchtime["HashSpan"] - launchtime["WasmInitSpan"] - launchtime["WasmLoadSpan"] \
                        - launchtime["WasmInstantiateSpan"] - launchtime["WasmExecutionSpan"]

launchtime

In [ ]:
# Compute the mean and std
launch_span = compute_mean_std(launchtime_raw["Size"], launchtime["LaunchSpan"], factor=0.001)
launch_span

In [ ]:
transition_span = compute_mean_std(launchtime_raw["Size"], launchtime["TransitionSpan"], factor=0.001)
transition_span

In [ ]:
memory_span = compute_mean_std(launchtime_raw["Size"], launchtime["MemorySpan"], factor=0.001)
memory_span

In [ ]:
hash_span = compute_mean_std(launchtime_raw["Size"], launchtime["HashSpan"], factor=0.001)
hash_span

In [ ]:
wasm_init_span = compute_mean_std(launchtime_raw["Size"], launchtime["WasmInitSpan"], factor=0.001)
wasm_init_span

In [ ]:
wasm_load_span = compute_mean_std(launchtime_raw["Size"], launchtime["WasmLoadSpan"], factor=0.001)
wasm_load_span

In [ ]:
wasm_instantiate_span = compute_mean_std(launchtime_raw["Size"], launchtime["WasmInstantiateSpan"], factor=0.001)
wasm_instantiate_span

In [ ]:
wasm_execution_span = compute_mean_std(launchtime_raw["Size"], launchtime["WasmExecutionSpan"], factor=0.001)
wasm_execution_span

In [ ]:
other_span = compute_mean_std(launchtime_raw["Size"], launchtime["OtherSpan"], factor=0.001)
other_span

In [ ]:
# Draw the area plot
area_plot_data = pd.DataFrame()
area_plot_data["Transition"] = transition_span["Mean"]
area_plot_data["Memory"] = memory_span["Mean"]
area_plot_data["Hash"] = hash_span["Mean"]
area_plot_data["WasmInit"] = wasm_init_span["Mean"]
area_plot_data["WasmLoad"] = wasm_load_span["Mean"]
area_plot_data["WasmInstantiate"] = wasm_instantiate_span["Mean"]
area_plot_data["WasmExecution"] = wasm_execution_span["Mean"]

area_plot_data.plot.bar(stacked=True) #.area()

In [ ]:
#
# Show the ratio of the elements
#

ratio = pd.DataFrame()
ratio["Transition"] = transition_span["Mean"] / launch_span["Mean"]
ratio["Memory"] = memory_span["Mean"] / launch_span["Mean"]
ratio["Hash"] = hash_span["Mean"] / launch_span["Mean"]
ratio["WasmInit"] = wasm_init_span["Mean"] / launch_span["Mean"]
ratio["WasmLoad"] = wasm_load_span["Mean"] / launch_span["Mean"]
ratio["WasmInstantiate"] = wasm_instantiate_span["Mean"] / launch_span["Mean"]
ratio["WasmExecution"] = wasm_execution_span["Mean"] / launch_span["Mean"]

ratio

In [ ]:
# Export CSV
#
# File format: experiment_name, mean, stddev

transition_span.drop(columns="Size").to_csv('../logs/launch-time/transition_formatted.csv')
memory_span.drop(columns="Size").to_csv('../logs/launch-time/memory_formatted.csv')
hash_span.drop(columns="Size").to_csv('../logs/launch-time/hash_formatted.csv')
wasm_init_span.drop(columns="Size").to_csv('../logs/launch-time/wasm_init_formatted.csv')
wasm_load_span.drop(columns="Size").to_csv('../logs/launch-time/wasm_load_formatted.csv')
wasm_instantiate_span.drop(columns="Size").to_csv('../logs/launch-time/wasm_instantiate_formatted.csv')
wasm_execution_span.drop(columns="Size").to_csv('../logs/launch-time/wasm_execution_formatted.csv')

In [ ]:
# Export LaTeX

ratio_median = ratio.median()
prefix = "LaunchTime"

def write_ratio(file, column_name, ratio):
    f.write(f"\\def\\bm{prefix}{column_name}Ratio{{{formatForLatex(ratio * 100)}}}\n")

f = open('../logs/launch-time/export.tex', "w")
write_ratio(f, "WasmInit", ratio_median["WasmInit"])
write_ratio(f, "WasmLoad", ratio_median["WasmLoad"])
write_ratio(f, "Memory", ratio_median["Memory"])
write_ratio(f, "Hash", ratio_median["Hash"])
f.close()